<a href="https://colab.research.google.com/github/Team7-Sept2021/test/blob/main/ML_Data_Science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, OrdinalEncoder

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error


from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier


from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
np.set_printoptions(precision=3)

%matplotlib inline
np.set_printoptions(precision=3, suppress=True)
pd.set_option('precision', 3)


In [65]:
df = pd.read_excel("mpg.data.xlsx")
df = df.dropna(axis=1, thresh= 200)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     406 non-null    int64  
 2   displayments  406 non-null    float64
 3   horsepower    400 non-null    float64
 4   weight        406 non-null    int64  
 5   acceleration  406 non-null    float64
 6   model year    406 non-null    int64  
 7   origin        406 non-null    int64  
 8   car name      406 non-null    object 
dtypes: float64(4), int64(4), object(1)
memory usage: 28.7+ KB


# Preprocessing

Missing Values

In [66]:
df_re_missing = df.dropna(axis = 0,subset=['mpg'])
median_value = df_re_missing['horsepower'].median()
df_re_missing['horsepower'] =df_re_missing['horsepower'].fillna(median_value)
df_prepro = df_re_missing.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [67]:
df_prepro['car_company'] = df_prepro['car name'].apply(lambda x: x.split()[0])

df_prepro.replace({'vokswagen': 'volkswagen',
            'vw': 'volkswagen', 
            'maxda': 'mazda', 
            'toyouta': 'toyota',
            'mercedes-benz': 'mercedes',
            'chevy': 'chevrolet',
            'chevroelt': 'chevrolet',
            'capri': 'ford'}, inplace=True)

df_prepro = df_prepro.drop(columns='car name')

In [68]:
dict_region = {1:'USA',2:'Europe',3:'Japan'}
df_prepro['origin'] = df_prepro['origin'].replace(dict_region)

In [69]:
df_prepro

,mpg,cylinders,displayments,horsepower,weight,acceleration,model year,origin,car_company
0,18.0,8,307.0,130.0,3504,12.0,70,USA,chevrolet
1,15.0,8,350.0,165.0,3693,11.5,70,USA,buick
2,18.0,8,318.0,150.0,3436,11.0,70,USA,plymouth
3,16.0,8,304.0,150.0,3433,12.0,70,USA,amc
4,17.0,8,302.0,140.0,3449,10.5,70,USA,ford
...,...,...,...,...,...,...,...,...,...
401,27.0,4,140.0,86.0,2790,15.6,82,USA,ford
402,44.0,4,97.0,52.0,2130,24.6,82,Europe,volkswagen
403,32.0,4,135.0,84.0,2295,11.6,82,USA,dodge
404,28.0,4,120.0,79.0,2625,18.6,82,USA,ford


In [41]:
#X =  df_prepro.drop(columns='mpg')
y =  df_prepro['mpg']
X = df_prepro.iloc[:,[2,3,4,5,6,8]]

In [42]:
seed = 42  # random seed for reproducability

# it handles both shuffling and splitting
# x_train, x_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=seed)

In [43]:
X_train.head()

,displayments,horsepower,weight,acceleration,model year,car_company
3,304.0,150.0,3433.0,12.0,70,amc
24,97.0,88.0,2130.0,14.5,70,datsun
384,91.0,68.0,2025.0,18.2,82,mazda
255,91.0,60.0,1800.0,16.4,78,honda
184,115.0,95.0,2694.0,15.0,75,audi


## If we keep 'origin' or 'car_company'

In [44]:
#OneHotEncoder  https://stackoverflow.com/questions/36631163/what-are-the-pros-and-cons-between-get-dummies-pandas-and-onehotencoder-sciki
X_train = X_train.reset_index()
encoder = OneHotEncoder( handle_unknown="ignore")
enc = pd.DataFrame(encoder.fit_transform(X_train[['car_company']]).toarray())
X_train = X_train.join(enc)
X_train.set_index('index', inplace=True)
#drop 'car_company','origin'
X_train = X_train.drop(columns=['car_company'])


In [45]:
X_test = X_test.reset_index()
enc = pd.DataFrame(encoder.transform(X_test[['car_company']]).toarray())
X_test = X_test.join(enc)
X_test.set_index('index', inplace=True)
#drop 'car_company','origin'
X_test = X_test.drop(columns=['car_company'])

# ML process

In [46]:
#StandardScaler  https://www.geeksforgeeks.org/normalization-vs-standardization/
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [47]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV

In [48]:
lassoReg = linear_model.Lasso()
linear = LinearRegression()
knr = KNeighborsRegressor()
regr = DecisionTreeRegressor()
svr = SVR()
rfr = RandomForestRegressor()
model = [lassoReg, linear, knr, regr, svr, rfr]
for i in model:
  i.fit(X_train, y_train)

  pred_test = i.predict(X_test)
  print(i)
  print(f'Mean Squared Error: {mean_squared_error(y_test, pred_test)}')  
  print(f'Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, pred_test))}')
  print(f'Mean Absolute Error: {mean_absolute_error(y_test, pred_test)}')
  print(f'R2:{r2_score(y_test,pred_test)}')
  print(f'Score: {i.score(X_test, y_test)}\n\n')

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
Mean Squared Error: 9.01046410819161
Root Mean Squared Error: 3.001743511393272
Mean Absolute Error: 2.3025660479178027
R2:0.8324147278535303
Score: 0.8324147278535303


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
Mean Squared Error: 3.0144113189461466e+23
Root Mean Squared Error: 549036548778.5077
Mean Absolute Error: 61384152262.498245
R2:-5.606491909642333e+21
Score: -5.606491909642333e+21


KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')
Mean Squared Error: 12.625715
Root Mean Squared Error: 3.5532682139123692
Mean Absolute Error: 2.59925
R2:0.7651748168670724
Score: 0.7651748168670724


DecisionTreeRegressor(ccp_alpha=

# KNeighborsRegressor - Grid

In [49]:
param_knr_grid = {'n_neighbors': np.arange(1,40, 2),
                  'algorithm':[ 'ball_tree', 'kd_tree', 'brute'],
                'weights': ['uniform', 'distance'],
                'leaf_size':np.arange(1, 100, 10),
                'p':np.arange(1, 4, 1)}


knr = KNeighborsRegressor()
gknr = GridSearchCV(knr, param_knr_grid, cv=KFold(n_splits=5,
                                              shuffle=True, random_state=0))
gknr.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': array([ 1, 11, 21, 31, 41, 51, 61, 71, 81, 91]),
                         'n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33,
       35, 37, 39]),
                         'p': array([1, 2, 3]),
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [57]:
pred_test = gknr.predict(X_test)
print(f'Mean Squared Error: {mean_squared_error(y_test, pred_test)}')  
print(f'Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, pred_test))}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, pred_test)}')
print(f'Score: {gknr.score(X_test, y_test)}')
print(f'R2:{r2_score(y_test,pred_test)}')

Mean Squared Error: 9.92700880836218
Root Mean Squared Error: 3.150715602583353
Mean Absolute Error: 2.467615198428947
Score: 0.8153679485569068
R2:0.8153679485569068


# RandomForestRegressor - Grid

In [51]:
param_rfr_grid = {'n_estimators': np.arange(10, 100, 20),
              'max_depth':np.arange(10, 110, 10),
              'max_features' : ['auto', 'sqrt'],
              'min_samples_split' : [2, 5, 10],
              'min_samples_leaf' : [1, 2, 4],
              'bootstrap' : [True, False]}


rfr = RandomForestRegressor()
grfr = GridSearchCV(rfr, param_rfr_grid, scoring='neg_mean_squared_error', cv=KFold(n_splits=5,
                                              shuffle=True, random_state=0))
grfr.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_est...
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'bootstrap

In [52]:
pred_test = grfr.predict(X_test)
print(f'Mean Squared Error: {mean_squared_error(y_test, pred_test)}')  
print(f'Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, pred_test))}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, pred_test)}')
print(f'Score: {grfr.score(X_test, y_test)}')
print(f'R2:{r2_score(y_test,pred_test)}')

Mean Squared Error: 4.70711220679012
Root Mean Squared Error: 2.169588026974273
Mean Absolute Error: 1.6281527777777776
Score: -4.70711220679012
R2:0.9124526028041399


# Lasso Regression

In [58]:
param_lassoReg_grid = {'alpha':np.arange(0, 1, 0.01)}


lassoReg = linear_model.Lasso()
glassoReg = GridSearchCV(lassoReg, param_lassoReg_grid, scoring='neg_mean_squared_error', cv=KFold(n_splits=5,
                                              shuffle=True, random_state=0))
glassoReg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1190.8802640346823, tolerance: 1.5798070393700787
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dis

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.0...
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='ne

In [59]:
pred_test = glassoReg.predict(X_test)
print(f'Mean Squared Error: {mean_squared_error(y_test, pred_test)}')  
print(f'Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, pred_test))}')
print(f'Mean Absolute Error: {mean_absolute_error(y_test, pred_test)}')
print(f'Score: {glassoReg.score(X_test, y_test)}')
print(f'R2:{r2_score(y_test,pred_test)}')

Mean Squared Error: 9.111527719958193
Root Mean Squared Error: 3.0185307220497513
Mean Absolute Error: 2.462257440594223
Score: -9.111527719958193
R2:0.8305350496617476


# Pipe

In [ ]:
#PCA